In [1]:
from datasets import load_dataset
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

SBERT_MODEL = "all-MiniLM-L6-v2"
from collections import Counter
import nltk
import re
from nltk import sent_tokenize

In [2]:
nsfw_dataset = load_dataset("jjmachan/NSFW-questions-inter-cleaned_df", split="train")
pro_social_dataset = load_dataset("allenai/prosocial-dialog", split="train")

Using custom data configuration jjmachan--NSFW-questions-inter-cleaned_df-90257bbb92c45b4c
Found cached dataset parquet (/home/shahul/.cache/huggingface/datasets/jjmachan___parquet/jjmachan--NSFW-questions-inter-cleaned_df-90257bbb92c45b4c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Using custom data configuration allenai--prosocial-dialog-ebbad39ca08b6d44
Found cached dataset json (/home/shahul/.cache/huggingface/datasets/allenai___json/allenai--prosocial-dialog-ebbad39ca08b6d44/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [3]:
nsfw_dataset

Dataset({
    features: ['title', 'subreddit', 'post_id', 'score', 'link_flair_text', 'is_self', 'over_18', 'upvote_ratio', 'is_question', 'C1', 'C2', 'C3', 'C4', 'C5'],
    num_rows: 12858
})

In [4]:
def match_rot_safetylabels(dataset):
    rots = [item["rots"] for item in dataset]
    safety_annotations = [item["safety_label"] for item in dataset]
    results = {}
    for rots, sfty in zip(rots, safety_annotations):
        for rot in rots:
            if rot not in results.keys():
                results[rot] = sfty
    return results

In [5]:
rot_sfty = match_rot_safetylabels(pro_social_dataset)
all_rots = list(set(rot_sfty.keys()))

In [6]:
def load_vectorizer(model=SBERT_MODEL):
    return SentenceTransformer(model)


def vectorize_text(model, texts):
    return model.encode(texts, show_progress_bar=True)

In [7]:
model = load_vectorizer()

In [8]:
rot_vector = vectorize_text(model, all_rots)

Batches:   0%|          | 0/3630 [00:00<?, ?it/s]

In [9]:
import scipy.spatial as sp
from collections import defaultdict
from tqdm import tqdm

THRESHOLD = 0.65


def match_query_rot(q, m):
    cosine_sim = 1 - sp.distance.cdist(q, m, "cosine")
    sim_indices = np.argwhere(cosine_sim >= THRESHOLD)
    return sim_indices

In [10]:
BATCH_SIZE = 100


def match_rot_post(dataset):
    dic = {}
    posts = [item["title"] for item in dataset]
    post_vector = vectorize_text(model, posts)
    for idx in tqdm(range(0, len(post_vector), BATCH_SIZE)):
        sim_indices = match_query_rot(post_vector[idx : idx + BATCH_SIZE], rot_vector)
        for post_idx, rot_idx in sim_indices:
            rot = all_rots[rot_idx]
            dic.update({dataset[int(post_idx) + idx]["post_id"]: {"rots": [rot], "safety_label": rot_sfty.get(rot)}})
    return dic

In [11]:
result_dict = match_rot_post(nsfw_dataset)

Batches:   0%|          | 0/402 [00:00<?, ?it/s]

100%|███████████████████████| 129/129 [04:03<00:00,  1.89s/it]


In [12]:
print("Turaround perc", len(result_dict) / len(nsfw_dataset) * 100)

Turaround perc 11.214807901695442


In [13]:
def filter_stopwords(example):
    stopwords = ["Ladies", "Women", "Gals", "Men", "guys"]
    regex = "".join([f"{word}(,)?|" for word in stopwords])
    example["title"] = re.sub(regex, "", example["title"], flags=re.IGNORECASE)
    return example

In [14]:
def add_rot_label(example):
    post_id = example["post_id"]
    if post_id in result_dict.keys():
        example["rots"] = result_dict.get(post_id)["rots"]
        example["safety_label"] = result_dict.get(post_id)["safety_label"]

    return example

In [15]:
def select_response(example):
    comments = comments_df[comments_df["post_id"] == example["post_id"]][
        ["C1", "C2", "C3", "C4", "C5"]
    ].values.tolist()[0]
    comments = [str(comment) for comment in comments]
    comments = [
        comment for comment in comments if (len(sent_tokenize(comment)) > 1) and (len(sent_tokenize(comment)) < 3)
    ]
    comments = [comment for comment in comments if re.search("(?P<url>https?://[^\s]+)", comment) is None]

    if comments:
        example["response"] = np.random.choice(comments, 1)[0]
        print(example["response"])

    return example

In [16]:
new_column = [[]] * len(nsfw_dataset)
nsfw_dataset = nsfw_dataset.add_column("rots", new_column)
new_column = [None] * len(nsfw_dataset)
nsfw_dataset = nsfw_dataset.add_column("safety_label", new_column)
new_column = ["None"] * len(nsfw_dataset)
nsfw_dataset = nsfw_dataset.add_column("response", new_column)

In [17]:
nsfw_dataset = nsfw_dataset.map(filter_stopwords)
nsfw_dataset = nsfw_dataset.map(add_rot_label)
nsfw_dataset = nsfw_dataset.filter(lambda example: example["safety_label"])

Loading cached processed dataset at /home/shahul/.cache/huggingface/datasets/jjmachan___parquet/jjmachan--NSFW-questions-inter-cleaned_df-90257bbb92c45b4c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a06a0f349e8c04ee.arrow


  0%|          | 0/12858 [00:00<?, ?ex/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

## Comments

In [18]:
post_ids = [item["post_id"] for item in nsfw_dataset]
comments_df = get_comments(post_ids)

In [22]:
nsfw_dataset = nsfw_dataset.map(select_response)

In [20]:
nsfw_dataset = nsfw_dataset.rename_columns({"title": "user"})
nsfw_dataset = nsfw_dataset.remove_columns(["C1", "C2", "C3", "C4", "C5", "link_flair_text", "score", "upvote_ratio"])

In [26]:
nsfw_dataset = nsfw_dataset.shuffle()
nsfw_dataset[35]

{'user': 'Why are condoms not always effective?',
 'subreddit': 'sexeducation',
 'post_id': 'le7znt',
 'is_self': True,
 'over_18': False,
 'is_question': True,
 'rots': ["It's good to use condoms to protect yourself during sex"],
 'safety_label': '__needs_caution__',
 'response': "They are absolutely weighing in the percentages of condoms breaking, slipping off, being used incorrectly, etc. \n\nIf a condom is used correctly, disposed of correctly, and doesn't break at any point, there's 0% chance of pregnancy as far as I'm aware."}

In [29]:
new_column = [True] * len(nsfw_dataset)
nsfw_dataset = nsfw_dataset.add_column("episode_done", new_column)

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

In [30]:
nsfw_dataset.push_to_hub("shahules786/prosocial-nsfw")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
